<a href="https://colab.research.google.com/github/dragonthereal/NYSE-files/blob/main/LarryConnersMeanReversionTradingStrategyUsingMovingAverages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports
!pip install alpaca_trade_api
import alpaca_trade_api as tradeapi
import pandas
import requests
import time
from datetime import date, timedelta
import math

In [ ]:
polygon_api = 'gYeZgQJrG8WXODzJou9U1ff3jKYU6VxB'
polygon_api_1 = '84pguIZC6cV1Sh04dsTclAK0auke0ttm'
polygon_api_2 = 'gSOBND4kEJE4WeQ3H3RmuKtaDghqIG_6'
polygon_api_3 = 'NVvcreaegiQSo30ayDnvienc2fGOlPvJ'
another_polygon_api_6 = 'vKKW5z8Rsw5B_UMQpfWypdWRS1S2iJwq'
another_polygon_api_7 = '7wA_DxZImf1f29l1Bk2JYhUupKvS_PhX'
another_polygon_api_8 = 'VcoSKMNQPpxK2nKi5039saYetbt4q8jR'

In [ ]:
alpha_vantage_api_eight = 'AG7X3IZH5TU0KF98'
alpha_vantage_api_nine = 'JEK2507WG40LMWPF'
alpha_vantage_api_ten = 'NN3RPDGYG33131HP'
alpha_vantage_api_eleven = "QAPIX4I33ZCQ3E9F"
alpha_vantage_api_thirteen = "G5BKWLG69KWONP66"
alpha_vantage_api_fourteen = "EW3MLHXOAUJTNIT2"
iex_cloud_api = 'pk_0f6d70e75f9141ec8d27ab2ea2988d94'

In [ ]:
#the function that allows us to call alphavantage many times
def alpha_vantage_call(function, symbol, additional_text, api_key, api_key_2):
    data = requests.get(f'https://www.alphavantage.co/query?function={function}&symbol={symbol}{additional_text}&apikey={api_key}').json()

    while 'Note' in data:
        data = requests.get(f'https://www.alphavantage.co/query?function={function}&symbol={symbol}{additional_text}&apikey={api_key_2}').json()
        
    
    while 'Error Message' in data:
        data = ''
        break
    return data

In [ ]:
#initializing all variables

#initializing date
today = date.today()
real_date = today.strftime("%Y-%m-%d")
real_date = str(real_date)
real_date = "2022-01-14"
#250 days ago
days_ago_250 = str(today - timedelta(days=250))

In [ ]:
def getting_data_polygon(stock_symbol, timespan, from_date, to_date, api_key, api_key_2):
    data = requests.get(f'https://api.polygon.io/v2/aggs/ticker/{stock_symbol}/range/1/{timespan}/{from_date}/{to_date}?adjusted=true&sort=desc&limit=5000&apiKey={api_key}').json()
    while 'error' in data:
        data = requests.get(f'https://api.polygon.io/v2/aggs/ticker/{stock_symbol}/range/1/{timespan}/{from_date}/{to_date}?adjusted=true&sort=desc&limit=5000&apiKey={api_key_2}').json()
        time.sleep(1.5)

    
    return data

In [ ]:
#authentication and connection details
api_key = 'PKM01ZLV6WJQDZWYXVN8'
api_secret = '2EiUN9ayUiknReCQOqH1YsQKDOPyVBUw7b3ooqO1'
base_url = 'https://paper-api.alpaca.markets'

# instantiate REST API
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')


# Get a list of all active assets.
active_assets = api.list_assets(status='active')



#iterating through all the stocks
for asset in active_assets:
    #stock symbol
    stock_symbol = str(asset.symbol)

 


   # historical volatility
    days_ago_150 = str(today-timedelta(days=150))
    historical_prices = getting_data_polygon(stock_symbol, 'day', days_ago_150, real_date, polygon_api, another_polygon_api_6)


    
    if historical_prices['resultsCount'] < 100:
        continue

    x = 0
    close_added_all = 0

    while x <= 99:
        close_price_on_day = float(historical_prices['results'][x]['c'])
        close_added_all += close_price_on_day
        x += 1

    average_close = close_added_all / 100

    x = 0

    sum_of_squared_difference = 0

    while x <= 99:
        close_price_on_day = float(historical_prices['results'][x]['c'])
        difference_between_average_close_and_close = average_close - close_price_on_day
        difference_between_average_close_and_close **= 2
        sum_of_squared_difference += difference_between_average_close_and_close
        x += 1

    sum_of_squared_difference /= 100
    historical_volatility = math.sqrt(sum_of_squared_difference)


    if historical_volatility > 30:
        condition_1 = True
    else:
        continue 


    
    # trying to see whether stock is above 5 dollars
    eod_data_last_stock_session = getting_data_polygon(stock_symbol, 'day', real_date, real_date, polygon_api_2, another_polygon_api_7)
    if eod_data_last_stock_session['status'] == 'DELAYED':
        continue
    
    close = float(eod_data_last_stock_session['results'][0]['c'])
    if close > 5 :
        condition_2 = True
    else:
        continue

    
    #getting last month's volume and checking whether it is above 250,000
    months_of_data = getting_data_polygon(stock_symbol, 'month', days_ago_150, real_date, polygon_api_3, another_polygon_api_8)
    last_month_volume = float(months_of_data['results'][0]['v'])

    if last_month_volume > 250_000:
        condition_3 = True
    else:
        continue

    
    #finding whether stock is above 200 day moving average
#finding 200 day moving average of past weeks
    day_moving_average_past_days_200 = alpha_vantage_call('EMA', stock_symbol, '&interval=daily&time_period=200&series_type=close', alpha_vantage_api_thirteen, alpha_vantage_api_fourteen)

    if not day_moving_average_past_days_200['Technical Analysis: EMA']:
        continue

#filtering today's 200 day moving average
    day_moving_average_200 = float(day_moving_average_past_days_200['Technical Analysis: EMA'][real_date]['EMA'])

#checking whether today's stock price is above 200 day moving average
    if close > day_moving_average_200:
        condition_4 = True
    else:
        continue

        
#making a function that allows us to call all these scenarios
    def scenarios_fast_below_slow(fastma, slowma):
        slowma_2_5 = slowma * 0.975
        slowma_10 = slowma * 0.9
    
        if fastma <= slowma_2_5 and fastma >= slowma_10:
            condition_5 = True
        else:
            condition_5 = False
        return condition_5

#getting EMA for 5, 10, 20, 50

#getting data for 5
    data_for_5 = alpha_vantage_call('EMA', stock_symbol,'&interval=daily&time_period=5&series_type=close', alpha_vantage_api_eight, alpha_vantage_api_nine)

#getting moving average 5 today
    moving_average_today_5 = float(data_for_5['Technical Analysis: EMA'][real_date]['EMA'])

#getting data for 10
    data_for_10 = alpha_vantage_call('EMA', stock_symbol,'&interval=daily&time_period=10&series_type=close', alpha_vantage_api_ten, alpha_vantage_api_eleven)

#getting moving average 10 today
    moving_average_today_10 = float(data_for_10['Technical Analysis: EMA'][real_date]['EMA'])

#getting data for 20
    data_for_20 = alpha_vantage_call('EMA', stock_symbol,'&interval=daily&time_period=20&series_type=close',  alpha_vantage_api_eight, alpha_vantage_api_nine)

#getting moving average 20 today
    moving_average_today_20 = float(data_for_20['Technical Analysis: EMA'][real_date]['EMA'])

#getting data for 50
    data_for_50 = alpha_vantage_call('EMA', stock_symbol,'&interval=daily&time_period=50&series_type=close', alpha_vantage_api_ten, alpha_vantage_api_eleven)

#getting moving average 50 today
    moving_average_today_50 = float(data_for_50['Technical Analysis: EMA'][real_date]['EMA'])

#testing the conditions
    if scenarios_fast_below_slow(moving_average_today_5, moving_average_today_10) == True:
        condition_5 = True
    elif scenarios_fast_below_slow(moving_average_today_5, moving_average_today_20) == True:
        condition_5 = True
    elif scenarios_fast_below_slow(moving_average_today_5, moving_average_today_50) == True:
        condition_5 = True
    elif scenarios_fast_below_slow(moving_average_today_10, moving_average_today_20) == True:
        condition_5 = True
    elif scenarios_fast_below_slow(moving_average_today_10, moving_average_today_50) == True:
        condition_5 = True
    else:
        continue
        

#price of stock at which to buy stock
    buy_stock_at = str(close * 0.96)

#the herculean task lol
#checking whether all conditions are true, then telling the owner to buy the stock
    if condition_1 == True and condition_2 == True and condition_3 == True and condition_4 == True and condition_5 == True:
        print('Buy ' + stock_symbol + ' at ' + buy_stock_at) 

LRCX condition_5 FAILED
REGN condition_5 FAILED
Buy SI at 132.624
Buy FDS at 410.0256
Buy SAIA at 265.536
ORLY condition_5 FAILED
